In [15]:
#Step 1:clone paper code

!git clone https://github.com/Coldog2333/SuperDialseg.git
%cd SuperDialseg

# install compatible libraries
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install transformers datasets scikit-learn evaluate tqdm




Cloning into 'SuperDialseg'...
remote: Enumerating objects: 298, done.
remote: Counting objects: 100% (298/298), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 298 (delta 89), reused 268 (delta 62), pack-reused 0 (from 0)
Receiving objects: 100% (298/298), 14.80 MiB | 7.22 MiB/s, done.
Resolving deltas: 100% (89/89), done.
/content/SuperDialseg/SuperDialseg
Looking in indexes: https://download.pytorch.org/whl/cu121


In [16]:
import zipfile, os

os.makedirs("/content/doc2dial", exist_ok=True)

zip_files = [
    "/content/drive/MyDrive/dialseg711-20251106T071822Z-1-001.zip",
    "/content/drive/MyDrive/superseg-20251106T071841Z-1-001.zip",
    "/content/drive/MyDrive/tiage-20251106T071942Z-1-001.zip",
    "/content/drive/MyDrive/zys-20251106T072010Z-1-001.zip"
]

for z in zip_files:
    with zipfile.ZipFile(z, 'r') as zip_ref:
        zip_ref.extractall("/content/doc2dial")

print("✅ All files extracted successfully!")



✅ All files extracted successfully!


In [17]:
os.listdir("/content/doc2dial")


['zys', 'dialseg711', 'tiage', 'superseg']

In [18]:
import os

for root, dirs, files in os.walk("/content/doc2dial"):
    print(root, files)


/content/doc2dial []
/content/doc2dial/zys ['segmentation_file_test.txt']
/content/doc2dial/dialseg711 ['segmentation_file_test.json', 'segmentation_file_test.txt']
/content/doc2dial/tiage ['segmentation_file_test.json', 'segmentation_file_validation.txt', 'segmentation_file_train.txt', 'segmentation_file_train.json', 'topic_segmentation_cr_validation.txt', 'topic_segmentation_cr_test.txt', 'topic_segmentation_cr_train.txt', 'segmentation_file_validation.json', 'segmentation_file_test.txt']
/content/doc2dial/superseg ['segmentation_file_test.json', 'segmentation_file_validation.txt', 'segmentation_file_train.txt', 'segmentation_file_train.json', 'topic_segmentation_cr_validation.txt', 'README.md', 'topic_segmentation_cr_test.txt', 'topic_segmentation_cr_train.txt', 'segmentation_file_validation.json', 'segmentation_file_test.txt']


In [19]:
import json

base_path = "/content/doc2dial/superseg"

with open(f"{base_path}/segmentation_file_train.json", "r") as f:
    train_data = json.load(f)

print("Top-level keys in training JSON:")
print(train_data.keys())

# Preview 1st few items safely
first_key = list(train_data.keys())[0]
print("\nExample from first key:")
# print(json.dumps(train_data[first_key][:1], indent=2))


Top-level keys in training JSON:
dict_keys(['dial_data'])

Example from first key:


In [20]:
# Extract actual dialogues from the dictionary
dialogues = list(train_data["dial_data"].values())

print("✅ Total dialogues in training data:", len(dialogues))

# Preview the first dialogue
print("\n🔹 First dialogue example:")
print(json.dumps(dialogues[0], indent=2))


#step 3 : dataset download complete


Streaming output truncated to the last 5000 lines.
    "turns": [
      {
        "da": "query_condition",
        "role": "user",
        "turn_id": 1,
        "utterance": "What kind of counseling is required before I receive my Federal Student Aid?",
        "topic_id": 0,
        "segmentation_label": 0
      },
      {
        "da": "query_condition",
        "role": "agent",
        "turn_id": 2,
        "utterance": "Have you heard of entrance counseling?",
        "topic_id": 1,
        "segmentation_label": 0
      },
      {
        "da": "response_positive",
        "role": "user",
        "turn_id": 3,
        "utterance": "Yes, but am not sure what it is.",
        "topic_id": 1,
        "segmentation_label": 0
      },
      {
        "da": "respond_solution",
        "role": "agent",
        "turn_id": 4,
        "utterance": "Entrance counseling is required before you can get your first Direct Subsidized or Unsubsidized Loan as an undergraduate, or your first Direct PLU

In [21]:
#To Check Which Type of data in 'dial_data' dataset

print(train_data.keys())  # Top-level keys
first_key = list(train_data.keys())[0]
print("\nFirst key data preview:")
print(json.dumps(train_data[first_key], indent=2)[:1000])


dict_keys(['dial_data'])

First key data preview:
{
  "superseg-v2": [
    {
      "dial_id": "0c0dd5a4a1dfb23135eec6b77bca2fd5",
      "turns": [
        {
          "da": "query_condition",
          "role": "user",
          "turn_id": 1,
          "utterance": "I'm looking for information regarding benefits planning, can you help me?",
          "topic_id": 0,
          "segmentation_label": 0
        },
        {
          "da": "query_condition",
          "role": "agent",
          "turn_id": 2,
          "utterance": "Are you planning for your own future?",
          "topic_id": 0,
          "segmentation_label": 0
        },
        {
          "da": "response_positive",
          "role": "user",
          "turn_id": 3,
          "utterance": "Yes I am",
          "topic_id": 0,
          "segmentation_label": 0
        },
        {
          "da": "respond_solution",
          "role": "agent",
          "turn_id": 4,
          "utterance": "You need to think about what your f

In [22]:
# Step 4 – Prepare Subset (SuperDialSeg data)
import json, random
from collections import Counter

# Load your train_data
base_path = "/content/doc2dial/superseg"
with open(f"{base_path}/segmentation_file_train.json", "r") as f:
    train_data = json.load(f)

# ✅ Step 1: Extract dialogues properly
dialogs = train_data["dial_data"]["superseg-v2"]

data_samples = []
for dialog in dialogs:
    for turn in dialog["turns"]:
        utt = turn.get("utterance", "").strip()
        label = int(turn.get("segmentation_label", 0))
        if utt:
            data_samples.append({"text": utt, "label": label})

print(f"✅ Total utterances extracted: {len(data_samples)}")
print("Label distribution:", Counter([d['label'] for d in data_samples]))

# ⚖️ Step 2: Balance the dataset (optional)
zeros = [d for d in data_samples if d['label'] == 0][:3000]
ones = [d for d in data_samples if d['label'] == 1][:3000]
balanced_data = zeros + ones
random.shuffle(balanced_data)

print("✅ After balancing:", Counter([d['label'] for d in balanced_data]))


✅ Total utterances extracted: 92151
Label distribution: Counter({0: 69888, 1: 22263})
✅ After balancing: Counter({1: 3000, 0: 3000})


In [23]:
# Step 5 — Tokenization
from transformers import BertTokenizerFast
from datasets import Dataset

# 1️. Load Tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# 2. Convert it into Huggingface Dataset
hf_dataset = Dataset.from_list(balanced_data)

# 3. Tokenization function
def preprocess(batch):
    return tokenizer(batch['text'], truncation=True, padding='max_length', max_length=128)
# 4. Apply preprocessing
tokenized = hf_dataset.map(preprocess, batched=True)

# 5. Train-test split
tokenized = tokenized.train_test_split(test_size=0.2, seed=42)

# 6. Format for PyTorch model
tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

print("✅ Tokenization done.")



print("✅ Tokenization and train-test split complete!")

print(tokenized)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

✅ Tokenization done.
✅ Tokenization and train-test split complete!
DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4800
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1200
    })
})


In [24]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [25]:

# Step 6 — Train Baseline Model (BERT)
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# 1. Load The Model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


 # Defining evaluation function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='binary', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': round(acc, 3),
        'f1': round(f1, 3),
        'precision': round(precision, 3),
        'recall': round(recall, 3)
    }

 # Reduce dataset and epochs
tokenized['train'] = tokenized['train'].select(range(300))
tokenized['test'] = tokenized['test'].select(range(100))

# Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    save_strategy="no",
    logging_steps=10,
    seed=42
)

# Trainer Setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['test'],
    compute_metrics=compute_metrics
)

# Train
trainer.train()

#  Evaluate
eval_results = trainer.evaluate(tokenized['test'])
print("\n✅ Evaluation Metrics:")
for k, v in eval_results.items():
    print(f"{k}: {v:.3f}")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,0.636500
20,0.493900
30,0.463800



✅ Evaluation Metrics:
eval_loss: 0.628
eval_accuracy: 0.650
eval_f1: 0.701
eval_precision: 0.577
eval_recall: 0.891
eval_runtime: 46.158
eval_samples_per_second: 2.166
eval_steps_per_second: 0.282
epoch: 1.000


In [26]:
#step 7 Accuray Metric

# 1) Get evaluation metrics (table)

# Print eval metrics (already printed by trainer.evaluate)
print(eval_results)

# If you want a nicer table:
import pandas as pd
df = pd.DataFrame([eval_results])
df = df.rename(columns=lambda c: c.replace("eval_", ""))
display(df)









{'eval_loss': 0.6277023553848267, 'eval_accuracy': 0.65, 'eval_f1': 0.701, 'eval_precision': 0.577, 'eval_recall': 0.891, 'eval_runtime': 46.1585, 'eval_samples_per_second': 2.166, 'eval_steps_per_second': 0.282, 'epoch': 1.0}


,loss,accuracy,f1,precision,recall,runtime,samples_per_second,steps_per_second,epoch
0,0.627702,0.65,0.701,0.577,0.891,46.1585,2.166,0.282,1.0


In [27]:
!pip install -U datasets huggingface_hub



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.0/515.0 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflict

In [28]:
# Step 8: Minor Modification

#load dataset
from datasets import load_dataset
import random

dataset = load_dataset("knkarthick/dialogsum")

# Generate samples with speaker tokens and balanced labels
def make_examples_with_speaker(dataset, n=200):
    data = []
    for d in dataset['train']:
        dialogue = d['dialogue']
        for i, utt in enumerate(dialogue.split('\n')):
            spk = "SPEAKER1" if i % 2 == 0 else "SPEAKER2"
            text = f"<{spk}> {utt.strip()}"
            # ⚖️ Balanced random labeling
            label = 0 if i % 2 == 0 else 1
            data.append({'text': text, 'label': label})
    random.shuffle(data)
    return data[:n]

modified_data = make_examples_with_speaker(dataset)
print("✅ Example sample:", modified_data[:3])


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

✅ Example sample: [{'text': '<SPEAKER2> #Person2#: just after 2 am.', 'label': 1}, {'text': '<SPEAKER1> #Person1#: That makes sense. I will do it.', 'label': 0}, {'text': '<SPEAKER1> #Person1#: Good!', 'label': 0}]


In [29]:
!pip install -U transformers huggingface-hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.1.2
    Uninstalling huggingface_hub-1.1.2:
      Successfully uninstalled huggingface_hub-1.1.2


In [30]:
# Again Tokenization for Modification
from transformers import AutoTokenizer
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Add speaker tokens
tokenizer.add_special_tokens({'additional_special_tokens': ['<SPEAKER1>', '<SPEAKER2>']})

def tokenize_fn(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=64)

hf_dataset = Dataset.from_list(modified_data)
print(f"✅ Original dataset size: {len(hf_dataset)} samples")

# Tokenize the dataset
tokenized = hf_dataset.map(tokenize_fn, batched=True)
print("✅ Tokenization completed!")

# ✅ Split dataset
tokenized = tokenized.train_test_split(test_size=0.2)
tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 🔹 Verify structure and sample
print(f"✅ Train size: {len(tokenized['train'])}, Test size: {len(tokenized['test'])}")
print("\n🔹 Example tokenized sample:")
print(tokenized['train'][0])


✅ Original dataset size: 200 samples


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

✅ Tokenization completed!
✅ Train size: 160, Test size: 40

🔹 Example tokenized sample:
{'label': tensor(0), 'input_ids': tensor([  101, 30522,  1001,  2711,  2487,  1001,  1024,  4596,  4269,  2012,
         1020,  1051,  1005,  5119,  1010,  2021,  3960,  1998,  2010,  6513,
         2097,  2272,  2058,  2011,  1019,  1024,  2382,  1012,  1045,  2342,
         1996,  3221,  2077,  2027,  2131,  2182,  1012,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}


In [31]:
modified_data = make_examples_with_speaker(dataset, n=200)


In [32]:
#for installing latest version of Transformer

!pip install -U transformers


In [33]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [34]:
# Model & Trainer setup
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.resize_token_embeddings(len(tokenizer))

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': round(acc, 3),
        'precision': round(precision, 3),
        'recall': round(recall, 3),
        'f1': round(f1, 3)
    }

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=3e-5,
    weight_decay=0.01,
    logging_steps=10,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['test'],
    compute_metrics=compute_metrics
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,0.693500
20,0.671700


TrainOutput(global_step=20, training_loss=0.6826248884201049, metrics={'train_runtime': 265.295, 'train_samples_per_second': 0.603, 'train_steps_per_second': 0.075, 'total_flos': 5262221107200.0, 'train_loss': 0.6826248884201049, 'epoch': 1.0})

In [35]:
# ✅ Step 5: Evaluate model
eval_results = trainer.evaluate(tokenized['test'])
print("\n✅ Evaluation Results:")
for k, v in eval_results.items():
    print(f"{k}: {v}")


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



✅ Evaluation Results:
eval_loss: 0.5939167141914368
eval_accuracy: 0.875
eval_precision: 0.792
eval_recall: 1.0
eval_f1: 0.884
eval_runtime: 16.333
eval_samples_per_second: 2.449
eval_steps_per_second: 0.306
epoch: 1.0


In [36]:
#Save MOdel and result

trainer.save_model("saved_model")
import pandas as pd
results = trainer.evaluate()

print(results)
pd.DataFrame([results]).to_csv("results.csv", index=False)

print("✅ Results saved in results.csv and model saved in saved_model/")




/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.5939167141914368, 'eval_accuracy': 0.875, 'eval_precision': 0.792, 'eval_recall': 1.0, 'eval_f1': 0.884, 'eval_runtime': 19.4131, 'eval_samples_per_second': 2.06, 'eval_steps_per_second': 0.258, 'epoch': 1.0}
✅ Results saved in results.csv and model saved in saved_model/
